In [59]:
%reload_ext autoreload
%autoreload 2
import sys, os, time
import utils
import pandas as pd
import numpy as np
from pathlib import Path

In [60]:
dir_dct = {
    "wrf_iTransformer": "long_term_forecast_wind_12_12_short_wrf_iTransformer_custom_ftMS_sl12_ll12_pl12_dm64_nh8_el3_dl1_df64_expand2_dc4_fc3_ebtimeF_dtTrue_Exp_0",
    "wrf_TimesNet": "long_term_forecast_wind_12_12_short_wrf_TimesNet_custom_ftMS_sl12_ll12_pl12_dm64_nh8_el3_dl1_df64_expand2_dc4_fc3_ebtimeF_dtTrue_Exp_0",
    "TimesNet": "long_term_forecast_wind_12_12_TimesNet_custom_ftMS_sl12_ll12_pl12_dm64_nh8_el3_dl1_df64_expand2_dc4_fc3_ebtimeF_dtTrue_Exp_0",
    "iTransformer": "long_term_forecast_wind_12_12_iTransformer_custom_ftMS_sl12_ll12_pl12_dm64_nh8_el3_dl1_df64_expand2_dc4_fc3_ebtimeF_dtTrue_Exp_0"
}

## RMSE & MAPE

In [61]:
# # LSTM model A
lstm_df = utils.load_metrics(
    dir="/Users/zhangxiangxian/wind_speed_prediction", 
    name="lstm", leadtime=12
)
lstm_df.loc[lstm_df["rmse"] > 10, "rmse"] = pd.NA
lstm_df.loc[lstm_df["mape"] < 0, "mape"] = pd.NA
lstm_df.loc[lstm_df["mape"] > 200, "mape"] = pd.NA

# LSTM + wrf
lstm_wrf_df = utils.load_metrics(
    dir="/Users/zhangxiangxian/wind_speed_prediction", 
    name="lstm_wrf", leadtime=12
)
lstm_wrf_df.loc[lstm_wrf_df["rmse"] > 10, "rmse"] = pd.NA
lstm_wrf_df.loc[lstm_wrf_df["mape"] < 0, "mape"] = pd.NA
lstm_wrf_df.loc[lstm_wrf_df["mape"] > 200, "mape"] = pd.NA

# # TimesNet
timesnet_df = utils.load_metrics(
    dir=dir_dct["TimesNet"],
    name="TimesNet_12"
)

# TimesNet + wrf
timesnet_wrf_df = utils.load_metrics(
    dir=dir_dct["wrf_TimesNet"],
    name="TimesNet_12_wrf"
)

# # iTransformer
itransformer_df = utils.load_metrics(
    dir=dir_dct["iTransformer"],
    name="iTransformer_12"
)

# iTransformer + wrf
itransformer_wrf_df = utils.load_metrics(
    dir=dir_dct["wrf_iTransformer"],
    name="iTransformer_12_wrf"
)


rmse_df = pd.DataFrame({
    "leadtime": lstm_df["leadtime"],
    "LSTM": lstm_df["rmse"],
    "LSTM_wrf": lstm_wrf_df["rmse"],
    "TimesNet": timesnet_df["rmse"],
    "TimesNet_wrf": timesnet_wrf_df["rmse"],
    "iTransformer": itransformer_df["rmse"],
    "iTransformer_wrf": itransformer_wrf_df["rmse"],
})

mape_df = pd.DataFrame({
    "leadtime": timesnet_df["leadtime"],
    "LSTM": lstm_df["mape"],
    "LSTM_wrf": lstm_wrf_df["mape"],
    "TimesNet": timesnet_df["mape"],
    "TimesNet_wrf": timesnet_wrf_df["mape"],
    "iTransformer": itransformer_df["mape"],
    "iTransformer_wrf": itransformer_wrf_df["mape"]
})


In [62]:
# Plot RMSE
rmse_fig = utils.plot_metrics(rmse_df, "rmse", ["LSTM", "LSTM_wrf", "TimesNet", "TimesNet_wrf", "iTransformer", "iTransformer_wrf"])
rmse_fig.show()
# rmse_fig.write_html(f"formosa_12_rmse_comparison.html")

# Plot MAPE
mape_fig = utils.plot_metrics(mape_df, "mape", ["LSTM", "LSTM_wrf", "TimesNet", "TimesNet_wrf", "iTransformer", "iTransformer_wrf"])
mape_fig.show()
# mape_fig.write_html(f"formosa_12_mape_comparison.html")

## Curves

In [63]:
prediction_length = 12

true_df = utils.load_true(dir_dct["iTransformer"])
lstm_df = utils.load_lstm_curve()
lstm_wrf_df = utils.load_lstm_curve(attr="wrf")
itransformer_wrf_df = utils.load_curve(dir_dct["wrf_iTransformer"], "iTransformer_wrf")
timesnet_wrf_df = utils.load_curve(dir_dct["wrf_TimesNet"], "TimesNet_wrf")
timesnet_df = utils.load_curve(dir_dct["TimesNet"], "TimesNet")
itransformer_df = utils.load_curve(dir_dct["iTransformer"], "iTransformer")

df = pd.concat([
        true_df, timesnet_wrf_df, itransformer_wrf_df, lstm_wrf_df, lstm_df, 
        timesnet_df, itransformer_df
    ], axis=1)

labels = ["iTransformer_wrf", "iTransformer", "TimesNet_wrf", "TimesNet", "lstm_pred", "lstm_wrf_pred"]
for i in range(1, prediction_length+1):
    fig = utils.plot_curve_by_leadtime(df, i, labels)
    fig.write_html(f"curve/leadtime{i}_comparison.html")
    time.sleep(1)

# for label in labels[1:]:
#     fig = utils.plot_curve_by_model(df, label)
#     fig.write_html(output_dir/f"{label}.html")
#     time.sleep(1)